In [6]:
from batid.services.guess_bdg_new import Guesser,GeocodeAddressHandler
from batid.models import Building
import csv
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
def row_to_input(line_number, row) -> dict:
    
    address_infos = [
        row['num_voie'],
        row['ind_rep'],
        row['nom_voie'],
        row['code_postal'],
        row['commune']
    ]
    
    address_infos = [i for i in address_infos if i]
    address = " ".join(address_infos)
    
    return {
        'ext_id': f"{row['num_arrete']}--ligne{line_number}",
        'address': address
    }
    

In [3]:
os.remove("guess.json")

with open("historique_insa_rnb.csv", "r") as f:
    
    reader = csv.DictReader(f)
    inputs = []
    
    for i, row in enumerate(list(reader)):
        inputs.append(row_to_input(i+2, row))
        
    
    guesser = Guesser()
    guesser.load_inputs(inputs)
    guesser.save_work_file('guess.json')

In [4]:
guesser = Guesser()


guesser.handlers = [GeocodeAddressHandler()]
guesser.guess_work_file('guess.json')

guesser.report()




-- Report --
Number of rows: 277
Number of match: 85 (30.69%)

-- match_reasons : absolute --
precise_address_match    85
Name: match_reason, dtype: int64

-- match_reasons : % --
precise_address_match    30.685921
Name: match_reason, dtype: float64

-- Inputs --
rows with ban_id: 164 (59.21%)


In [5]:
guesser.display_unmatched()

    match_reason     finished_steps        input_ext_id  \
130         None  [geocode_address]   2022-95--ligne132   
273         None  [geocode_address]  2023-137--ligne275   
48          None  [geocode_address]    2021-79--ligne50   
74          None  [geocode_address]    2022-25--ligne76   
218         None  [geocode_address]   2023-68--ligne220   
148         None  [geocode_address]  2022-126--ligne150   
261         None  [geocode_address]  2023-124--ligne263   
36          None  [geocode_address]    2021-53--ligne38   
213         None  [geocode_address]   2023-63--ligne215   
216         None  [geocode_address]   2023-66--ligne218   

                                         input_address      input_ban_id  \
130          12 RUE DU CHEVALIER ROZE 13002 MARSEILLE                NaN   
273                96 RUE JEAN MERMOZ 13008 MARSEILLE   13208_4774_00096   
48       3 AVENUE AMBROISE CROIZAT 13110 PORT-DE-BOUC   13077_0025_00003   
74                     33 RUE VITALIS 13005 MA

In [8]:
bdgs = Building.objects.filter(addresses__id='13208_4774_00096')
print(bdgs.count())

5
